## Strategy Three

In [ ]:
""" Loading packages """
import pandas as pd
import numpy as np
from datetime import datetime
import sklearn as skl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from matplotlib.pyplot import figure
import scipy.stats as ss

%matplotlib inline
sns.set_style('whitegrid')

import math
import random
import pyomo.opt
from pyomo.core import Var
from pyomo.environ  import *

import time

""" Loading Battery Class """
from battery import Battery

## Data Preparation
Here:
* We read the data
* Fix the date to have a **Date format** which is DD/MM/YY HH:MM
* Check if there are missing values and fill them up (incase)
* Set 'timesteps' or 'Datetime' to be the index

In [ ]:
""" Read the data """

df = pd.read_csv("load_and_dam_data.csv")
df.columns = ['Datetime', 'load_kW', 'dam_price']
df['Datetime'] = [datetime.strptime(x, '%d/%m/%y %H:%M') for x in df['Datetime']]
pd.set_option('display.max_rows', None)

# filling data using the median function
def fillmissingvalues(data):
    median = data.median()
    data.fillna(median, inplace = True)
    
    return data 
df = fillmissingvalues(df)
#setting the datetime as an index
dataset = df
dataset = df.set_index("Datetime")
dataset.index = pd.to_datetime(dataset.index)
print(dataset.info())

### Series of functions incase they're needed to be used:
* A function that normalizes the data to the range you want

In [ ]:
""" normalizing the data """
def theDataNormalizer(data):
    scaler = MinMaxScaler(feature_range = (0, 1))
    data_scaled = scaler.fit_transform(data)
    
    return data_scaled
#test
data_norm = theDataNormalizer(dataset.iloc[:,:])

Which day is going to be plotted and analysed? 

In [ ]:
d=21 #day number

# Step One: Day ahead bid commitment:

In [ ]:
''' Hourly '''
dataset_hourly = pd.DataFrame()

# load (kW) resampling from 15 min interval to 1 hour interval
dataset_hourly['load_kW'] = dataset.iloc[:,0].resample('H').sum()

#average DAM prices resampling from 15 min interval to 1 hour interval
dataset_hourly['dam_price'] = dataset.iloc[:,1].resample('H').mean()

#random uniform error between -1 and 1
dataset_hourly['randError'] = ss.uniform.rvs(size=745, loc =-1, scale=2, random_state = 1995)
#random Normal error between -1 and 1
#dataset_hourly['randError'] = ss.norm.rvs(size=745, loc =0, scale=1/3, random_state = 1995)
# random gamma distribution error between -1 and 1
#dataset_hourly['randError'] = ss.gamma.rvs(a = 2, size=745, loc = 0, scale=0.17, random_state= 1995)

# Synthetic Day-ahead Forecast 
dataset_hourly['SyntheticLoadForecast'] = dataset_hourly['load_kW'] + dataset_hourly['randError']

''' By 15 Minutes '''

#random normal error between -1 and 1
#dataset['randError'] = ss.norm.rvs(size=2977, loc =0, scale=1/3, random_state = 1995)
#random uniform error between -1 and 1
#dataset['randError'] = ss.uniform.rvs(size=2977, loc =-1, scale=2, random_state = 1995)
# random gamma distribution error between -1 and 1
dataset['randError'] = ss.gamma.rvs(a = 2, size=2977, loc = 0, scale=0.17, random_state= 1995)

# Synthetic Day-ahead Forecast 
dataset['SyntheticLoadForecast'] = dataset['load_kW'] + dataset['randError']


In [ ]:
# creating 10 scenarios for the forecasted deviations
num_hour_timesteps = 24
num_timesteps = 96
num_scenarios = 50
devs = []
devs_15 = []
for i in range(num_scenarios):
        # Uniform
        #devs.append(ss.uniform.rvs(size=num_hour_timesteps, loc =-1, scale=2))
        #devs_15.append(ss.uniform.rvs(size=num_timesteps, loc =-1, scale=2))
        # Normal
        #devs.append(ss.norm.rvs(size=num_hour_timesteps, loc =0, scale=1/3))
        #devs_15.append(ss.norm.rvs(size=num_timesteps, loc =0, scale=1/3))
        # Gamma
        devs.append(ss.gamma.rvs(a = 2, size=num_hour_timesteps, loc = 0, scale=0.17)) # gamma 
        devs_15.append(ss.gamma.rvs(a = 2, size=num_timesteps, loc = 0, scale=0.17))


dev_table = pd.DataFrame(devs).transpose()
dev_15_table = pd.DataFrame(devs_15).transpose()

### Day Index column

Here an extra column is added to differentiate days between eachother.
There exist 31 days with the current dataset and each day has 24 slots so each day will have like:
(same applies for 96 15-min slots)

Day 1: t=1 dayIndex = 1,
       t=2 dayIndex = 1,
       ...
       t=24 dayIndex = 1
...
...

Day 20: t=1 dayIndex = 20,
        t=2 dayIndex = 20,
        ...
        t=24 dayIndex = 20
...
...

In [ ]:
dataset_daily =  dataset.iloc[:,:].resample('D').sum()
dataset_hourly['dayIndex'] = None
dataset['dayIndex'] = None
dayIndex = []
dayIndex_15 = []
for i in range(1, len(dataset_daily)+1): 
    for j in range(num_hour_timesteps):
        dayIndex.append(i)
    for k in range(num_timesteps):
        dayIndex_15.append(i)
        
for a in range(len(dataset_hourly)):
    dataset_hourly.iloc[a,4] = dayIndex[a]
    
for b in range(len(dataset)):    
    dataset.iloc[b,4] = dayIndex_15[b]

Given that there:

* A synthetic forecast of the load
* The parameters of the uniformly randomized error (load forecast error) 
* The day ahead market price (Assumed to have this)

The point is to find an optimal way to choose a day-ahead market commitment which minimizes the expectation of the energy cost.

In [ ]:
# making np arrays (it is unnecessary but just makes my life easier)
p_forecast = dataset_hourly.loc[dataset_hourly['dayIndex'] == d].SyntheticLoadForecast.values
dam_price = dataset_hourly.loc[dataset_hourly['dayIndex'] == d].dam_price.values
deviation = dev_table

p_forecast_15 = dataset.loc[dataset['dayIndex'] == d].SyntheticLoadForecast.values
dam_price_15 = dataset.loc[dataset['dayIndex'] == d].dam_price.values
deviation_15 = dev_15_table
        
# Duration of a market dispatch time interval
MarketTime = 1 # 1 = 1 hour, 0.25 = 15 minutes
# Imbalance cost
imb_cost = 0.02
prob_s = 1/num_scenarios

#creating dummy data for battery
b_data = {'init_SOC': [0.6], # initial State of Charge
          'capacity':  [50.0], # Battery capacity [kWh]
          'char_P_Limit': [1.0], # Max charging power [kW]
          'dis_P_Limit': [1.0], # Max discharging power [kW]
          'char_P_Eff': [0.95], # charging power efficiency
          'dis_P_Eff': [0.95], # discharging power efficiency
          'batt_min_SOC':[0] # minimum SOC the battery should reach
          }

batteryData = pd.DataFrame (b_data, columns = ['capacity','init_SOC','char_P_Limit','dis_P_Limit', 'char_P_Eff', 'dis_P_Eff', 'batt_min_SOC'])

# use battery class to store information
batt = Battery(state_of_charge = batteryData.loc[:,'init_SOC'][0],
           capacity = batteryData.loc[:,'capacity'][0],
           charging_power_limit = batteryData.loc[:,'char_P_Limit'][0],
           discharging_power_limit = batteryData.loc[:,'dis_P_Limit'][0],
           charging_efficiency = batteryData.loc[:,'char_P_Eff'][0],
           discharging_efficiency = batteryData.loc[:,'dis_P_Eff'][0],
           min_SOC = batteryData.loc[:,'batt_min_SOC'][0])

In [ ]:
""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" --- Model --- """""""""""""""""
""""""""""""""""""""""""""""""""""""""""""""
m_hourly = ConcreteModel()

""" Sets of periods """
m_hourly.t = RangeSet(0, len(dataset_hourly.loc[dataset_hourly['dayIndex'] == d])-1)
m_hourly.t_restrict = RangeSet(0, len(dataset_hourly.loc[dataset_hourly['dayIndex'] == d])-2)
m_hourly.t_beg = RangeSet(0, len(dataset_hourly.loc[dataset_hourly['dayIndex'] == d])-1, 23)

""" Set of Batteries """
m_hourly.b = RangeSet(0, len(batteryData)-1)

""" Set of deviation scenarios """
m_hourly.s = RangeSet(0, len(deviation.columns)-1)



""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" Variables """""""""""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""

#Here, the variables with their bounds that the model will use are defined. 
#These are what the model will try to optimize at every timestep

### The committed power constraint:
m_hourly.p_committed = Var(m_hourly.t)

# boolean character: either charging or discharging 
m_hourly.bool_char = Var(m_hourly.t * m_hourly.b * m_hourly.s, within = Binary)

m_hourly.powercharge = Var(m_hourly.t * m_hourly.b * m_hourly.s, bounds=(0, batt.charging_power_limit), within=NonNegativeReals)
m_hourly.powerdischarge = Var(m_hourly.t * m_hourly.b * m_hourly.s, bounds=(0,batt.discharging_power_limit), within=NonNegativeReals)
m_hourly.b_energy = Var(m_hourly.t * m_hourly.b * m_hourly.s, bounds=(0, batt.capacity))
m_hourly.batterypower = Var(m_hourly.t * m_hourly.b * m_hourly.s, bounds=(-batt.discharging_power_limit,batt.discharging_power_limit))
m_hourly.SOC = Var(m_hourly.t * m_hourly.b * m_hourly.s, bounds=(0, 1))

m_hourly.p = Var(m_hourly.t * m_hourly.b * m_hourly.s, bounds=(0,99999))
m_hourly.q = Var(m_hourly.t * m_hourly.b * m_hourly.s, bounds=(0,99999))


In [ ]:
""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""" Day Ahead Objective Function """""""""
""""""""""""""""""""""""""""""""""""""""""""""""
def obj_dayahead_energy_cost(m):
    return sum(sum(prob_s * sum(((dam_price[t] * (p_forecast[t] + m_hourly.batterypower[t,b,s] - dev_table.iloc[t,s])) + (imb_cost * m_hourly.p[t,b,s]) + (imb_cost * m_hourly.q[t,b,s])) * MarketTime for t in m_hourly.t) for b in m_hourly.b) for s in m_hourly.s)

m_hourly.dayahead_total_energy_cost = Objective(rule=obj_dayahead_energy_cost,sense=minimize)

In [ ]:
""" All battery constraint """

#battery power constraint
def battery_power_constraint(m,t,b,s):
    return m_hourly.batterypower[t,b,s] == (m_hourly.powercharge[t,b,s] - m_hourly.powerdischarge[t,b,s])
# Battery energy at time t and scenario s
def b_energy_t_constraint(m,t,b,s):
    return m_hourly.b_energy[t,b,s] == m_hourly.SOC[t,b,s]*batt.capacity
# boolean constraints
def bool_control_charging(m,t,b,s):
    return ((m_hourly.powercharge[t,b,s]) <= (batt.charging_power_limit * m_hourly.bool_char[t,b,s]))
def bool_control_discharge(m,t,b,s):
    return ((m_hourly.powerdischarge[t,b,s]) <= (batt.discharging_power_limit * (1 - m_hourly.bool_char[t,b,s])))
# SOC Evolution Constraint
def soc_evolution(m, t_r, b,s):
    return m_hourly.b_energy[t_r+1,b,s] == m_hourly.b_energy[t_r,b,s] + (MarketTime*m_hourly.powercharge[t_r,b,s] * batt.charging_efficiency - (MarketTime*m_hourly.powerdischarge[t_r,b,s] / batt.discharging_efficiency))
# SOC initial state value at start of every day
def init_state_day_SOC(m,t,b,s):
    return m_hourly.b_energy[t,b,s] == batt.state_of_charge*batt.capacity
# minimum battery SOC constraint
def min_bat_soc(m,t,b,s):
    return m_hourly.b_energy[t,b,s] >= batt.min_SOC * batt.capacity
def p_comLargerthanzero(m,t):
    return m_hourly.p_committed[t] >= 0
def abs_constraint(m,t,b,s):
    return p_forecast[t] - dev_table.iloc[t,s] + m_hourly.batterypower[t,b,s] - m_hourly.p_committed[t] + m_hourly.p[t,b,s] - m_hourly.q[t,b,s] == 0


m_hourly.init_state_day_SOC = Constraint(m_hourly.t_beg, m_hourly.b, m_hourly.s, rule=init_state_day_SOC)
#m_hourly.min_SOC_const = Constraint(m_hourly.t, m_hourly.b, rule=min_bat_soc)
m_hourly.p_com = Constraint(m_hourly.t, rule=p_comLargerthanzero)
m_hourly.battery_power = Constraint(m_hourly.t, m_hourly.b, m_hourly.s, rule=battery_power_constraint)
m_hourly.b_energy_constraint = Constraint(m_hourly.t, m_hourly.b, m_hourly.s, rule=b_energy_t_constraint)
m_hourly.Batt_boolean_charge = Constraint(m_hourly.t, m_hourly.b, m_hourly.s, rule=bool_control_charging)
m_hourly.Batt_boolean_discharge = Constraint(m_hourly.t, m_hourly.b, m_hourly.s, rule=bool_control_discharge)
m_hourly.soc_evol = Constraint(m_hourly.t_restrict, m_hourly.b, m_hourly.s, rule=soc_evolution)
m_hourly.abs_constraint = Constraint(m_hourly.t, m_hourly.b, m_hourly.s, rule = abs_constraint)

In [ ]:
""""""""""""""" Start Timer """""""""""""""
total_start_time = time.time()
""""""""""""""""""""""""""""""""""""""""""

In [ ]:
""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" Solver to be used """""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""
# set the path to the solver
opt = SolverFactory("cbc", executable="/Users/alaakamalcheaib/Desktop/UPC STUFF/ampl.macos64/cbc")

""""""""""""""""""""""""""""""""""""""""""
""" Solving the optimization problem: """
""""""""""""""""""""""""""""""""""""""""""

""""""""""""""" Start Timer """""""""""""""
hour_start_time = time.time()
""""""""""""""""""""""""""""""""""""""""""

results = opt.solve(m_hourly, tee=False)

""""""""""""""" End Timer """""""""""""""
hour_elapsed = time.time() - hour_start_time
print ('Time elapsed', hour_elapsed)
""""""""""""""""""""""""""""""""""""""""""

In [ ]:
ind_DA_hour = list(m_hourly.t)
p_committed_hour = list(m_hourly.p_committed.get_values().values())

res_pyomo_DA_hour = pd.DataFrame({'timesteps': ind_DA_hour,'p_committed':p_committed_hour,})
res_pyomo_DA_hour = res_pyomo_DA_hour.set_index("timesteps")

### Reshaping from 1D array to Matrix (24,10)

In [ ]:
pd.set_option('display.max_rows', None)
SOC_hour_values = list(m_hourly.SOC.get_values().values())
SOC_hour_values = np.array(SOC_hour_values).reshape(24,num_scenarios)

## From Hourly values to 15-minute interval values

In [ ]:
# Changing from (24,10) to (96,10)
SOC_values = []
for i in range(24):
    for j in range(4):
        SOC_values.append(SOC_hour_values[i])
        
SOC_values = np.array(SOC_values).reshape(96,num_scenarios)
SOC_table = pd.DataFrame(SOC_values)

""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" -- Model - """""""""""""""""
""""""""""""""""""""""""""""""""""""""""""""
m = ConcreteModel()

""" Sets of periods """
m.t = RangeSet(0, len(dataset.loc[dataset['dayIndex'] == d])-1)
m.t24 = RangeSet(0, 23)
m.t_restrict = RangeSet(0, len(dataset.loc[dataset['dayIndex'] == d])-2)
m.t_beg = RangeSet(0, len(dataset.loc[dataset['dayIndex'] == d])-1, 95)
m.t_byFour = RangeSet(0, len(dataset.loc[dataset['dayIndex'] == d])-1, 4)

""" Set of Batteries """
m.b = RangeSet(0, len(batteryData)-1)

""" Set of deviation scenarios """
m.s = RangeSet(0, len(deviation_15.columns)-1)

""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" Variables """""""""""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""

#Here, the variables with their bounds that the model will use are defined. 
#These are what the model will try to optimize at every timestep

### The committed power constraint:
m.p_committed = Var(m.t)

# boolean character: either charging or discharging 
m.bool_char = Var(m.t * m.b * m.s, within = Binary)

m.powercharge = Var(m.t * m.b * m.s, bounds=(0, batt.charging_power_limit), within=NonNegativeReals)
m.powerdischarge = Var(m.t * m.b * m.s, bounds=(0,batt.discharging_power_limit), within=NonNegativeReals)
m.b_energy = Var(m.t * m.b * m.s, bounds=(0, batt.capacity))
m.batterypower = Var(m.t * m.b * m.s, bounds=(-batt.discharging_power_limit,batt.discharging_power_limit))
m.SOC = Var(m.t * m.b * m.s, bounds=(0, 1))

m.p = Var(m.t * m.b * m.s, bounds=(0,99999))
m.q = Var(m.t * m.b * m.s, bounds=(0,99999))


In [ ]:
""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""" Day Ahead Objective Function """""""""
""""""""""""""""""""""""""""""""""""""""""""""""
def obj_dayahead_energy_cost(m):
    return sum(sum(prob_s * sum(((dam_price_15[t] * (p_forecast_15[t] + m.batterypower[t,b,s] - dev_15_table.iloc[t,s])) + (imb_cost * m.p[t,b,s]) + (imb_cost * m.q[t,b,s])) * MarketTime for t in m.t) for b in m.b) for s in m.s)


m.dayahead_total_energy_cost = Objective(rule=obj_dayahead_energy_cost,sense=minimize)

In [ ]:
""" All battery constraint """

#battery power constraint
def battery_power_constraint(m,t,b,s):
    return m.batterypower[t,b,s] == (m.powercharge[t,b,s] - m.powerdischarge[t,b,s])
# Battery energy at time t and scenario s
def b_energy_t_constraint(m,t,b,s):
    return m.b_energy[t,b,s] == m.SOC[t,b,s]*batt.capacity
# boolean constraints
def bool_control_charging(m,t,b,s):
    return ((m.powercharge[t,b,s]) <= (batt.charging_power_limit * m.bool_char[t,b,s]))
def bool_control_discharge(m,t,b,s):
    return ((m.powerdischarge[t,b,s]) <= (batt.discharging_power_limit * (1 - m.bool_char[t,b,s])))
# SOC Evolution Constraint
def soc_evolution(m, t_r, b,s):
    return m.b_energy[t_r+1,b,s] == m.b_energy[t_r,b,s] + (MarketTime*m.powercharge[t_r,b,s] * batt.charging_efficiency - (MarketTime*m.powerdischarge[t_r,b,s] / batt.discharging_efficiency))
# SOC initial state value at start of every day
def init_state_day_SOC(m,t,b,s):
    return m.b_energy[t,b,s] == batt.state_of_charge*batt.capacity
# Imbalance value constraint
#def imbalance_equation(m, t, b, s):
    #return m.imbalance[t] == (dev_table.iloc[t,s] + m.batterypower_dev[t,b])
# minimum battery SOC constraint
def min_bat_soc(m,t,b,s):
    return m.b_energy[t,b,s] >= batt.min_SOC * batt.capacity
def p_comLargerthanzero(m,t):
    return m.p_committed[t] >= 0
def SOC_foreveryhour(m,t,b,s):
    return m.SOC[t,b,s] == SOC_table.iloc[t,s]
def abs_constraint(m,t,b,s):
    return p_forecast_15[t] - dev_15_table.iloc[t,s] + m.batterypower[t,b,s] - m.p_committed[t] + m.p[t,b,s] - m.q[t,b,s] == 0


m.init_state_day_SOC = Constraint(m.t_beg, m.b, m.s, rule=init_state_day_SOC)
#m.min_SOC_const = Constraint(m.t, m.b, rule=min_bat_soc)
m.p_com = Constraint(m.t, rule=p_comLargerthanzero)
m.battery_power = Constraint(m.t, m.b, m.s, rule=battery_power_constraint)
m.b_energy_constraint = Constraint(m.t, m.b, m.s, rule=b_energy_t_constraint)
m.Batt_boolean_charge = Constraint(m.t, m.b, m.s, rule=bool_control_charging)
m.Batt_boolean_discharge = Constraint(m.t, m.b, m.s, rule=bool_control_discharge)
m.soc_evol = Constraint(m.t_restrict, m.b, m.s, rule=soc_evolution)
m.soc_forEveryHour = Constraint(m.t_byFour, m.b, m.s, rule=SOC_foreveryhour)
m.abs_constraint = Constraint(m.t, m.b, m.s, rule = abs_constraint)

In [ ]:
""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" Solver to be used """""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""
# set the path to the solver
opt = SolverFactory("cbc", executable="/Users/alaakamalcheaib/Desktop/UPC STUFF/ampl.macos64/cbc")

""""""""""""""""""""""""""""""""""""""""""
""" Solving the optimization problem: """
""""""""""""""""""""""""""""""""""""""""""

""""""""""""""" Start Timer """""""""""""""
min15_start_time = time.time()
""""""""""""""""""""""""""""""""""""""""""

results = opt.solve(m, tee=True)

""""""""""""""" End Timer """""""""""""""
min15_elapsed = time.time() - min15_start_time
print ('Time elapsed', min15_elapsed)
""""""""""""""""""""""""""""""""""""""""""

In [ ]:
""""""""""""""" End Timer """""""""""""""
elapsed = time.time() - total_start_time
print ('Time elapsed', elapsed)
""""""""""""""""""""""""""""""""""""""""""

In [ ]:
ind_DA = list(m.t)
p_committed = list(m.p_committed.get_values().values())
res_pyomo_DA = pd.DataFrame({'timesteps': ind_DA,'p_committed':p_committed,})
res_pyomo_DA = res_pyomo_DA.set_index("timesteps")

### Graph of committed power vs forecasted power vs DAM prices at every time t for a single day

In [ ]:
resdata = {'forenorm': dataset.loc[dataset['dayIndex'] == d].SyntheticLoadForecast,
        'damnorm': dataset.loc[dataset['dayIndex'] == d].dam_price,
        'comnorm': res_pyomo_DA.p_committed.values}

dfres = pd.DataFrame (resdata, columns = ['forenorm','damnorm','comnorm'])

#normalized
dfnorm = theDataNormalizer(dfres)

Committed power vs forecasted power vs DAM prices at every time t for a single day

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,8))
ax2 = ax1.twinx()
lns1 = ax1.plot(res_pyomo_DA.index, dfres.iloc[:,0], "-s", color='#4daf4a',
                linewidth=3, label = 'forecasted power') # forecasted power
lns2 = ax1.plot(res_pyomo_DA.index, dfres.iloc[:,2], color ='#377eb8',
                linewidth=3, linestyle="-", label = 'committed power') # committed power
lns3 = ax2.plot(res_pyomo_DA.index, dfres.iloc[:,1], color = '#ff7f00',
                linewidth=3, linestyle= (0, (5, 3)), label = 'DAM prices') # DAM prices

# added these three lines
lns = lns1+lns2+lns3
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc = 9, prop={'size': 18})

ax1.set_xlabel('timesteps (96)', fontsize = 20)
ax1.set_ylabel('power (kW)', fontsize = 20)
ax2.set_ylabel('DAM prices', fontsize = 20)

#defining display layout 
plt.tight_layout()

# Save figure
#fig.savefig('Results/StrategyThree/Day-ahead/Day{0}/Norm_committed_vs_forecast_vs_DAM_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyThree/Day-ahead/Day{0}/Uniform_committed_vs_forecast_vs_DAM_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyThree/Day-ahead/Day{0}/gamma_committed_vs_forecast_vs_DAM_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')

# show graph
plt.show()

### Residuals plotting
Residuals = forecasted power - committed power

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,8))

ax1.plot(res_pyomo_DA.index, dfres.iloc[:,0] - dfres.iloc[:,2], "-H", color='black', linewidth=3, label = 'residuals') # residuals


ax1.set_xlabel('timesteps (96)', fontsize = 20)
ax1.set_ylabel('residuals', fontsize = 20)

#defining display layout 
plt.tight_layout()

# legend
ax1.legend(loc = 0, prop={'size': 18})


# Save figure
#fig.savefig('Results/StrategyThree/Day-ahead/Day{0}/Norm_DAresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyThree/Day-ahead/Day{0}/Uniform_DAresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyThree/Day-ahead/Day{0}/gamma_DAresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')


# show graph
plt.show()

### Residuals distribution

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,8))
kwargs = dict(hist_kws={'alpha':.6}, kde_kws={'linewidth':2})
sns.distplot(dfres.iloc[:,0] - dfres.iloc[:,2], color="dodgerblue", label="Residuals Distribution",  **kwargs)
plt.legend(loc = 0, prop={'size': 18});

# Save figure
#fig.savefig('Results/StrategyThree/Day-ahead/Day{0}/Norm_dist_DAresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyThree/Day-ahead/Day{0}/Uniform_dist_DAresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyThree/Day-ahead/Day{0}/gamma_dist_DAresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')

# Step Two: The next day comes
In this stage, it's assumed that the forecast is available and the day has arrived. Here we have the "commited power" which is the power that has been generated in the bidding stage. There is the "actual power" which is the "actual load power" and the "batter power". The difference between actual and commited power is named as "imbalance".

The point here is to optimize the schedule of the battery usage in order to minimize the expectation of the energy cost.

First the data being used for the optimization of the battery usage.

First, the optimization problem will be written using **Pyomo** optimization package:
* Creating the Model model
* setting the indexes:
    * model.t is the set which contains all the timesteps
    * model.t_restrict is the set that the same m.t but it eliminates the last index
    * model.t_beg is the set that contains values from m.t but incremented by 96 (0, 96, 192,...)

Second, The variables with their bounds that the model will use are defined. These are what the model will try to optimize at every timestep

Then, what is defined:
* Objective function: minimize energy cost
* constraints:
    * battery_power_constraint: batterypower = powercharge - powerdischarge
    * imbalance_equation: imbalance = p_actualLoad + batterypower - p_commited
    * b_energy_t_constraint: battery energy at every time t (0 <= battery energy <= battery capacity)
    * bool_control_charging and bool_control_discharging: these constraints tell the battery that it can only charge OR discharge during one time period
    * soc_evolution: calculates the battery energy at the next timestep t where t_retsrict is being used
    * init_state_day_SOC: initial state of charge at start of every day (which is 60%) where t_beg is being used

In [ ]:
p_actualLoad = dataset.loc[dataset['dayIndex'] == d].load_kW.values
p_commit = res_pyomo_DA.iloc[:,0]
dam_price = dataset.loc[dataset['dayIndex'] == d].dam_price.values

""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" --- Model --- """""""""""""""""
""""""""""""""""""""""""""""""""""""""""""""
model = ConcreteModel()

""" Sets of periods """
model.t = RangeSet(0, len(dataset.loc[dataset['dayIndex'] == d])-1)
model.t_restrict = RangeSet(0, len(dataset.loc[dataset['dayIndex'] == d])-2)
model.t_beg = RangeSet(0, len(dataset.loc[dataset['dayIndex'] == d])-1, 95)

""" Set of Batteries """
model.b = RangeSet(0,len(batteryData)-1)


""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" Variables """""""""""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""

#Here, the variables with their bounds that the model will use are defined. These are what the model will try to optimize at every timestep

# Amount of electricity (dis)charged of battery at time t  (kW)
model.powercharge = Var(model.t * model.b, bounds=(0, batt.charging_power_limit), within=NonNegativeReals)
model.powerdischarge = Var(model.t * model.b, bounds=(0,batt.discharging_power_limit), within=NonNegativeReals)
model.batterypower = Var(model.t * model.b, bounds=(-batt.discharging_power_limit,batt.discharging_power_limit))

# Energy stored in battery at time t [kWh]
model.b_energy = Var(model.t * model.b, bounds=(0, batt.capacity))
model.SOC = Var(model.t * model.b, bounds=(0, 1))

# boolean character: either charging or discharging 
model.bool_char = Var(model.t * model.b, within = Binary)

model.imbalance = Var(model.t)

model.p = Var(model.t * model.b, bounds=(0,99999))
model.q = Var(model.t * model.b, bounds=(0,99999))

""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" Objective Function """""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""
def obj_energy_cost(m):
    return sum(sum(((dam_price[t] * p_actualLoad[t]) + (dam_price[t] * (model.batterypower[t,b])) + (imb_cost * model.p[t,b]) + (imb_cost * model.q[t,b]))*MarketTime for t in m.t) for b in m.b)  

model.total_energy_cost = Objective(rule=obj_energy_cost,sense=minimize)

""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" Constraints """""""""""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""
#battery power constraint
def battery_power_constraint(m,t,b):
    return model.batterypower[t,b] == (model.powercharge[t,b] - model.powerdischarge[t,b])
#Absolute value constraint
def imbalance_equation(m, t ,b):
    return model.imbalance[t] == (p_actualLoad[t] - p_commit[t] + model.batterypower[t,b])
# Battery energy at time t
def b_energy_t_constraint(m,t,b):
    return model.b_energy[t,b] == model.SOC[t,b]*batt.capacity
# boolean constraints
def bool_control_charging(m,t,b):
    return ((model.powercharge[t,b]) <= (batt.charging_power_limit * model.bool_char[t,b]))
def bool_control_discharge(m,t,b):
    return ((model.powerdischarge[t,b]) <= (batt.discharging_power_limit * (1 - model.bool_char[t,b])))
# SOC Evolution Constraint
def soc_evolution(m, t_r, b):
    return model.b_energy[t_r+1,b] == model.b_energy[t_r,b] + (MarketTime*model.powercharge[t_r,b] * batt.charging_efficiency - (MarketTime*model.powerdischarge[t_r,b] / batt.discharging_efficiency))
# SOC initial state value at start of every day
def init_state_day_SOC(m,t,b):
    return model.b_energy[t, b] == batt.state_of_charge*batt.capacity
# minimum battery SOC constraint
def min_bat_soc(m, t, b):
    return model.b_energy[t, b] >= batt.min_SOC * batt.capacity
# Absolute function constraint
def absolute_constraint(model,t,b):
    return model.imbalance[t] + model.p[t,b] - model.q[t,b] == 0

model.battery_power = Constraint(model.t,model.b,rule=battery_power_constraint)
model.b_energy_constraint = Constraint(model.t,model.b,rule=b_energy_t_constraint)
model.imb_eq = Constraint(model.t,model.b,rule=imbalance_equation)
model.Batt_boolean_charge = Constraint(model.t, model.b, rule=bool_control_charging)
model.Batt_boolean_discharge = Constraint(model.t, model.b, rule=bool_control_discharge)
model.soc_evol = Constraint(model.t_restrict, model.b, rule=soc_evolution)
model.init_state_day_SOC = Constraint(model.t_beg, model.b, rule=init_state_day_SOC)
model.min_SOC_const = Constraint(model.t, model.b, rule=min_bat_soc)
model.abs_eq = Constraint(model.t, model.b, rule = absolute_constraint)

In [ ]:
""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" Solver to be used """""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""
# set the path to the solver
#opt = SolverFactory("couenne", executable="/Users/alaakamalcheaib/Desktop/UPC STUFF/ampl.macos64/couenne")
opt = SolverFactory("cbc", executable="/Users/alaakamalcheaib/Desktop/UPC STUFF/ampl.macos64/cbc")

""""""""""""""""""""""""""""""""""""""""""
""" Solving the optimization problem: """
""""""""""""""""""""""""""""""""""""""""""

""""""""""""""" Start Timer """""""""""""""
total_start_time = time.time()
""""""""""""""""""""""""""""""""""""""""""

results = opt.solve(model, tee=False)

""""""""""""""" End Timer """""""""""""""
elapsed = time.time() - total_start_time
print ('Time elapsed', elapsed)
""""""""""""""""""""""""""""""""""""""""""

In [ ]:
ind_RT = list(model.t)
powerc_RT = list(model.powercharge.get_values().values())
powerd_RT = list(model.powerdischarge.get_values().values())
bpow_RT = list(model.batterypower.get_values().values())
bsoc_RT = list(model.SOC.get_values().values())
imb_RT = list(model.imbalance.get_values().values())
bEn_RT = list(model.b_energy.get_values().values())

res_pyomo_RT = pd.DataFrame(
    {'timesteps': ind_RT,
     'power_charge': powerc_RT,
     'power_discharge': powerd_RT,
     'battery_power': bpow_RT,
     'battery_SOC': bsoc_RT,
     'imbalance': imb_RT,
     'battery_energy': bEn_RT})

res_pyomo_RT = res_pyomo_RT.set_index("timesteps")

In [ ]:
#normalized
dfres_RT = pd.DataFrame(
    {'forenorm': dataset.loc[dataset['dayIndex'] == d].SyntheticLoadForecast,
     'damnorm': dataset.loc[dataset['dayIndex'] == d].dam_price,
     'comnorm': res_pyomo_DA.p_committed.values,
     'loadnorm': dataset.loc[dataset['dayIndex'] == d].load_kW,
     'batSOCnorm': bsoc_RT
    })
dfnorm_RT = theDataNormalizer(dfres_RT)

Committed vs Actual load power vs battery SOC at every time t for a single day

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,8))
ax2 = ax1.twinx()

lns1 = ax1.plot(dfres_RT.index, dfres_RT.iloc[:,2], "-", color= '#377eb8',
                linewidth=3, label = 'committed power')
lns2 = ax1.plot(dfres_RT.index, dfres_RT.iloc[:,3], "-s", color= '#e41a1c',
                linewidth=3, label = 'actual load power')
lns3 = ax2.plot(dfres_RT.index, dfres_RT.iloc[:,4], color= '#a65628',
                linewidth=3, linestyle= (0, (5, 3)), label = 'battery SOC')


# added these three lines
lns = lns1+lns2+lns3
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc = 9, prop={'size': 18})

ax1.set_xlabel('timesteps (96)', fontsize=20)
ax1.set_ylabel('power (kW)', fontsize=20)
ax2.set_ylabel('Battery SOC', fontsize=20)

#defining display layout 
plt.tight_layout()
# Save figure
#fig.savefig('Results/StrategyThree/Battery-operation/Day{0}/Norm_forecast__committed_realLoad_BatterySOC_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyThree/Battery-operation/Day{0}/Uniform_forecast__committed_realLoad_BatterySOC_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyThree/Battery-operation/Day{0}/gamma_forecast__committed_realLoad_BatterySOC_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')

# show graph
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,8))
ax1.plot(res_pyomo_DA.index, dfres_RT.iloc[:,2] - dfres_RT.iloc[:,3], "-H", color='black', linewidth=3, label = 'residuals') # residuals

ax1.legend(loc = 0, prop={'size': 18})

ax1.set_xlabel('timesteps (96)', fontsize = 20)
ax1.set_ylabel('residuals', fontsize = 20)

#defining display layout 
plt.tight_layout()

# Save figure
#fig.savefig('Results/StrategyThree/Battery-operation/Day{0}/Norm_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyThree/Battery-operation/Day{0}/Uniform_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyThree/Battery-operation/Day{0}/gamma_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')


# show graph
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,8))
kwargs = dict(hist_kws={'alpha':.6}, kde_kws={'linewidth':2})
sns.distplot(dfres_RT.iloc[:,3] - dfres_RT.iloc[:,2], color="dodgerblue", label="Residuals Distribution",  **kwargs)
plt.legend(loc = 0, prop={'size': 18});

# Save figure
#fig.savefig('Results/StrategyThree/Battery-operation/Day{0}/Norm_dist_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyThree/Battery-operation/Day{0}/Uniform_dist_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyThree/Battery-operation/Day{0}/gamma_dist_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')

In [ ]:
# saving the results in a csv file
result_BatOpt_RT = np.asarray(res_pyomo_RT)
#np.savetxt('Results/StrategyThree/Battery-operation/Day{0}/power_committed_Day{0}.csv'.format(d), result_BatOpt_RT, delimiter=',')

In [ ]:
pd.set_option('display.max_rows', False)
res_pyomo_RT

In [ ]:
imbalance_cost = res_pyomo_RT.iloc[:,4] * imb_cost

dam_price = dataset.loc[dataset['dayIndex'] == d].dam_price
bidding_cost = []

for t in range(len(dataset.loc[dataset['dayIndex'] == d])):
    result = res_pyomo_DA.iloc[t,0] * dam_price.iloc[t]
    bidding_cost.append(result)

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,8))
ax2 = ax1.twinx()

lns1 = ax1.plot(dfres_RT.index, bidding_cost, "-", color= '#984ea3',linewidth=3, label = "bidding cost")
lns2 = ax2.plot(dfres_RT.index, imbalance_cost, "-s", color= '#f781bf',linewidth=3, label = "imbalance cost")

# added these three lines
lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc = 3, prop={'size': 18})

ax1.set_xlabel('timesteps (96)', fontsize=18)
ax1.set_ylabel('bidding costs', fontsize=18)
ax2.set_ylabel('Imbalances', fontsize=18)

#defining display layout 
plt.tight_layout()

# Save figure
#fig.savefig('Results/StrategyThree/Battery-operation/Day{0}/Norm_bidding_vs_imbalance_cost_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyThree/Battery-operation/Day{0}/Uniform_bidding_vs_imbalance_cost_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyThree/Battery-operation/Day{0}/gamma_bidding_vs_imbalance_cost_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')

# show graph
plt.show()

bidding vs imbalance cost at every time t for a single day

In [ ]:
# Strategy Three
total_bidding_cost = 0
total_imbalance_cost = 0
total_overall_cost = 0

for t in range(len(dataset.loc[dataset['dayIndex'] == d])):
    total_bidding_cost += bidding_cost[t]
    total_imbalance_cost += abs(imbalance_cost[t])
    total_overall_cost += dam_price.iloc[t] * (p_commit[t] + res_pyomo_RT.iloc[t,4]) + abs(imbalance_cost[t])


    
total_overall_cost,total_bidding_cost,total_imbalance_cost

## Horizontal Bar Chart:
To compare the bidding cost, imbalance cost and total overall cost from each strategy

In [ ]:
# Day = 21
# [strat 1, strat 2, strat 3 ,forecast with battery, forecast no battery] at 50 scenarios
strats = ['Strat One', 'Strat Two', 'Strat Three','Forecast w/ battery', 'Forecast No Battery']
bars = ['Total_cost', 'Total_bid', 'total_imbalance']
colors = ['#1D2F6F', '#8390FA', '#6EAF46']


Totals_21_norm = [np.array([2.3055989, 2.498514, 2.560604, 2.80784513, 3.3268939]), # total overall cost values
                  np.array([2.5412306, 2.647839, 2.651744, 2.851028, 2.851028]), # total bidding cost values
                  np.array([0.186632, 0.070222, 0.071818, 0.10188026, 0.527922])] # total imbalance cost


Totals_21_Uniform = [np.array([2.424508, 2.4912798, 2.5824272, 2.83151218, 3.66768515]), # total overall cost values
                     np.array([2.331638, 2.45280979, 2.6240203, 2.5904109, 2.5904109]), # total bidding cost values
                     np.array([0.154517, 0.0598999, 0.047524, 0.180988, 0.86871359])] # total imbalance cost


Totals_norm_average = [np.array([3.458147, 3.60681, 3.609142, 3.80027, 4.197532]), # total overall cost values
                       np.array([3.379708, 3.69294, 3.623535, 3.70302, 3.796169]), # total bidding cost values
                       np.array([0.132346, 0.05816, 0.082800, 0.09989, 0.522459])] # total imbalance cost

Totals_Uniform_average = [np.array([3.562276, 3.66779, 3.689741, 3.884071, 4.197532]), # total overall cost values
                          np.array([3.695029, 4.00769, 3.991186, 3.766234, 3.796169]), # total bidding cost values
                          np.array([0.206854, 0.19675, 0.150393, 0.150413, 0.522459])] # total imbalance cost

Totals_gamma_average = [np.array([3.036598, 3.3989, 3.2612, 3.693, 4.10726]), # total overall cost values
                          np.array([3.15119, 3.9041, 3.4366, 5.0081, 5.0081]), # total bidding cost values
                          np.array([0.11383, 0.2025, 0.0832, 0.6368, 0.6651])] # total imbalance cost

###### Guassian - Day 21

In [ ]:
fig, ax = plt.subplots(1, figsize = (16, 8))

plt.barh(strats, Totals_21_norm[0], color = colors[0])
plt.barh(strats, Totals_21_norm[1], left = Totals_21_norm[0], color = colors[1])
plt.barh(strats, Totals_21_norm[2], left = Totals_21_norm[0]+Totals_21_norm[1], color = colors[2])
plt.xlabel('values', fontsize=20)
plt.ylabel('Strategies', fontsize=20)
plt.legend(bars, prop={'size': 20})

#plt.title('Total cost vs bidding cost vs imbalance cost across different strategies', fontsize=22)
#fig.savefig('Results/totals_normal_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')

plt.show()


###### Uniform - Day 21

In [ ]:
fig, ax = plt.subplots(1, figsize = (16, 8))

ax.barh(strats, Totals_21_Uniform[0], color = colors[0])
ax.barh(strats, Totals_21_Uniform[1], left = Totals_21_Uniform[0], color = colors[1])
ax.barh(strats, Totals_21_Uniform[2], left = Totals_21_Uniform[0]+Totals_21_Uniform[1], color = colors[2])
plt.xlabel('values')
plt.ylabel('Strategies')
plt.legend(bars, prop={'size': 20})
#plt.title('Total cost vs bidding cost vs imbalance cost across different strategies')

#fig.savefig('Results/totals_Uniform_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')

plt.show()

##### Gaussian - average 

In [ ]:
fig, ax = plt.subplots(1, figsize = (16, 8))

plt.barh(strats, Totals_norm_average[0], color = colors[0])
plt.barh(strats, Totals_norm_average[1], left = Totals_norm_average[0], color = colors[1])
plt.barh(strats, Totals_norm_average[2], left = Totals_norm_average[0]+Totals_norm_average[1], color = colors[2])
plt.xlabel('values', fontsize=20)
plt.ylabel('Strategies', fontsize=20)
plt.legend(bars, prop={'size': 20})

#plt.title('Total cost vs bidding cost vs imbalance cost across different strategies', fontsize=22)
#fig.savefig('Results/totals_normal_average.png', dpi=300, bbox_inches='tight')

plt.show()

##### Uniform - average

In [ ]:
fig, ax = plt.subplots(1, figsize = (16, 8))

plt.barh(strats, Totals_Uniform_average[0], color = colors[0])
plt.barh(strats, Totals_Uniform_average[1], left = Totals_Uniform_average[0], color = colors[1])
plt.barh(strats, Totals_Uniform_average[2], left = Totals_Uniform_average[0]+Totals_Uniform_average[1], color = colors[2])
plt.xlabel('values', fontsize=20)
plt.ylabel('Strategies', fontsize=20)
plt.legend(bars, prop={'size': 20})

#plt.title('Total cost vs bidding cost vs imbalance cost across different strategies', fontsize=22)
#fig.savefig('Results/totals_Uniform_average.png', dpi=300, bbox_inches='tight')

plt.show()

##### gamma - average

In [ ]:
fig, ax = plt.subplots(1, figsize = (16, 8))

plt.barh(strats, Totals_gamma_average[0], color = colors[0])
plt.barh(strats, Totals_gamma_average[1], left = Totals_gamma_average[0], color = colors[1])
plt.barh(strats, Totals_gamma_average[2], left = Totals_gamma_average[0]+Totals_gamma_average[1], color = colors[2])
plt.xlabel('values', fontsize=20)
plt.ylabel('Strategies', fontsize=20)
plt.legend(bars, prop={'size': 20})

#plt.title('Total cost vs bidding cost vs imbalance cost across different strategies', fontsize=22)
#fig.savefig('Results/totals_gamma_average.png', dpi=300, bbox_inches='tight')

plt.show()

### Step One

In [ ]:
# Step One
fig = plt.figure(figsize=(12,6))
obj_strat_1 = [4.282,4.291,4.328,4.318,5.236]
obj_strat_2 = [4.363,4.406,4.416,4.409,4.4207]
obj_strat_3 = [4.744,4.76,4.8,4.796,4.910]

scenarios = ['10', '50' ,'100', '200', '500']

# plot lines
plt.plot(scenarios, obj_strat_1,  "-o", label = "Strategy 1")
plt.plot(scenarios, obj_strat_2,  "-o", label = "Strategy 2")
plt.plot(scenarios, obj_strat_3,  "-o", label = "Strategy 3")
plt.xlabel("Number of Scenarios")
plt.ylabel("Objective function")
plt.legend()

#fig.savefig('Results/DA_numScenarios_vs_ObjFunc_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#plt.show()

In [ ]:
fig = plt.figure(figsize=(12,6))
time_strat_1 = [1.96,25.32,122.95,665.01,4373.12]
time_strat_2 = [0.99,5.63,14.18,60.53,339.91]
time_strat_3 = [5.47,24.46,65.21,352.76,3628.61]

scenarios = ['10', '50' ,'100', '200', '500']

# plot lines
plt.plot(scenarios, time_strat_1,  "-o", label = "Strategy 1")
plt.plot(scenarios, time_strat_2,  "-o", label = "Strategy 2")
plt.plot(scenarios, time_strat_3,  "-o", label = "Strategy 3")
plt.xlabel("Number of Scenarios")
plt.ylabel("Computation time")
plt.legend()

#fig.savefig('Results/DA_numScenarios_vs_CompTime_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#plt.show()

### Step Two:

In [ ]:
# Step Two
fig = plt.figure(figsize=(12,6))
obj_strat_1 = [4.0811341,4.04548,4.05868,4.05714, 4.24227]
obj_strat_2 = [4.161426,4.15616,4.16552,4.1663922, 4.16538]
obj_strat_3 = [4.18505,4.1715,4.181405,4.17924, 4.198]

scenarios = ['10', '50' ,'100', '200', '500']

# plot lines
plt.plot(scenarios, obj_strat_1,  "-o", label = "Strategy 1")
plt.plot(scenarios, obj_strat_2,  "-o", label = "Strategy 2")
plt.plot(scenarios, obj_strat_3,  "-o", label = "Strategy 3")
plt.xlabel("Number of Scenarios")
plt.ylabel("Objective function")
plt.legend()

#fig.savefig('Results/RT_numScenarios_vs_ObjFunc_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#plt.show()

In [ ]:
fig = plt.figure(figsize=(12,6))
time_strat_1 = [0.19218,0.17785,0.18417096,0.1882, 0.20803]
time_strat_2 = [0.139769,0.145486,0.1317,0.1364, 0.1482]
time_strat_3 = [0.192702,0.21532,0.2008,0.1934, 0.1989]

scenarios = ['10', '50' ,'100', '200', '500']

# plot lines
plt.plot(scenarios, time_strat_1,  "-o", label = "Strategy 1")
plt.plot(scenarios, time_strat_2,  "-o", label = "Strategy 2")
plt.plot(scenarios, time_strat_3,  "-o", label = "Strategy 3")
plt.xlabel("Number of Scenarios")
plt.ylabel("Computation time")
plt.legend()

#fig.savefig('Results/RT_numScenarios_vs_CompTime_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#plt.show()